In [1]:
import pickle
import glob

In [2]:
def readPerWord(file):
    words = []
    f = open(file, 'r') #open file
    for line in f:
        words = line.split(' ') #tiap satu baris split per kata
    
    f.close()
    
    words[len(words)-1] = words[len(words)-1].replace('\n',' ')

    return words

In [3]:
def extFeature(words):
    listOfDictFeature = []
    i = 0
    flag = 0 #0 = not tag, 1 = person, 2 = location, 3 = organization
    for i in range(len(words)): #iterasi dari awal tiap kata
        dictFeature = {}
        #cek apakah word tersebut tag
        if flag == 0: 
            if words[i] == '<person>':
                flag = 1
                continue
            if words[i] == '<location>':
                flag = 2
                continue
            if words[i] == '<organization>':
                flag = 3
                continue
            else:
                dictFeature = inputToDict(flag, words, i)
        elif flag == 1:
            if words[i] == '</person>':
                flag = 0
                continue
            dictFeature = inputToDict(flag, words, i)
        elif flag == 2:
            if words[i] == '</location>':
                flag = 0
                continue
            dictFeature = inputToDict(flag, words, i)
        elif flag == 3:
            if words[i] == '</organization>':
                flag = 0
                continue
            dictFeature = inputToDict(flag, words, i)
        
        listOfDictFeature.append(dictFeature)
        
    return listOfDictFeature

def inputToDict(flag, words, i):
    dictFeature = {}
    
    dictFeature['index'] = i    
    dictFeature['first_upper_case'] = isFirstUpper(words[i])
    dictFeature['prev_word'] = isEdge(i, words)
    dictFeature['word'] = words[i]
    
    if flag == 1:
        dictFeature['jenis'] = 'person'
    elif flag == 2:
        dictFeature['jenis'] = 'location'
    elif flag == 3:
        dictFeature['jenis'] = 'organization'
    else:
        dictFeature['jenis'] = 'o'
    
    return dictFeature

def isFirstUpper(word):
    if word[0].isupper():
        return 'y'
    else:
        return 'n'
    
def isEdge(i, words):
    if i == 0:
        return 'null'
    elif isTag(words[i-1]):
        return words[i-2]
    else:
        return words[i-1]

def isTag(word):
    tagWord = ['<person>','</person>','<location>','</location>','<organization>','</organization>']
    if word in tagWord:
        return True
    else:
        return False

In [4]:
def readFile(directory):
    listFile = glob.glob(directory+"/*.txt")
    return listFile

In [8]:
# words = readPerWord('berita1.txt')
# listOfDictFeature = extFeature(words)
# for i in listOfDictFeature:
#     print(i)
listOfDictFeatureAll = []
listOfFile = readFile("training")
for file in listOfFile:
    words = readPerWord(file)
    listOfDictFeature = extFeature(words)
    listOfDictFeatureAll.extend(listOfDictFeature)

In [10]:
for i in listOfDictFeatureAll:
    print(i)

{'index': 0, 'first_upper_case': 'y', 'prev_word': 'null', 'word': 'Menteri', 'jenis': 'o'}
{'index': 1, 'first_upper_case': 'y', 'prev_word': 'Menteri', 'word': 'Pertahanan', 'jenis': 'o'}
{'index': 3, 'first_upper_case': 'y', 'prev_word': 'Pertahanan', 'word': 'Prabowo', 'jenis': 'person'}
{'index': 4, 'first_upper_case': 'y', 'prev_word': 'Prabowo', 'word': 'Subianto', 'jenis': 'person'}
{'index': 6, 'first_upper_case': 'n', 'prev_word': 'Subianto', 'word': 'dikabarkan', 'jenis': 'o'}
{'index': 7, 'first_upper_case': 'n', 'prev_word': 'dikabarkan', 'word': 'akan', 'jenis': 'o'}
{'index': 8, 'first_upper_case': 'n', 'prev_word': 'akan', 'word': 'ke', 'jenis': 'o'}
{'index': 10, 'first_upper_case': 'y', 'prev_word': 'ke', 'word': 'Amerika', 'jenis': 'location'}
{'index': 11, 'first_upper_case': 'y', 'prev_word': 'Amerika', 'word': 'Serikat', 'jenis': 'location'}
{'index': 12, 'first_upper_case': 'n', 'prev_word': 'Serikat', 'word': '(AS)', 'jenis': 'location'}
{'index': 14, 'first_upp

In [11]:
pickle_out = open("training.pickle","wb")
pickle.dump(listOfDictFeatureAll, pickle_out)
pickle_out.close()